In [ ]:
from facenet_pytorch import InceptionResnetV1
from pathlib import Path
from PIL import Image
import numpy as np
import pandas as pd
import torch
from fpt.path import DTFR, FACE
from fpt.path import get_face_image_path_from_series

In [ ]:
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
_df_face = pd.read_csv(
    DTFR / "df_aihub_sample_face.csv", index_col="key", dtype={"folder_name": object}
)
df_file = pd.read_csv(DTFR / "df_aihub_sample_file.csv", index_col="key")
df_face = _df_face.join(df_file, on="key")

In [ ]:
def preprocess(face_image_path: Path) -> torch.Tensor:
    pil_img = Image.open(face_image_path)
    arr_img = np.array(pil_img, dtype=np.float32)
    img_cropped = torch.from_numpy(arr_img)
    img_cropped = img_cropped.unsqueeze(0)
    img_cropped = torch.permute(img_cropped, (0, 3, 1, 2))
    return img_cropped

In [ ]:
face_image_path = get_face_image_path_from_series(df_face.iloc[0])
img_cropped = preprocess(face_image_path)

In [ ]:
# Calculate embedding (unsqueeze to add batch dimension)
resnet.classify = False
img_embedding = resnet(img_cropped)
img_embedding.shape

In [ ]:
# Or, if using for VGGFace2 classification
resnet.classify = True
img_probs = resnet(img_cropped)
img_probs.shape